In [70]:
from PIL import Image
import numpy as np
import cv2
from u2net import *
import torch
from torchvision import transforms

In [71]:
from  ipypixano import Pixano

In [72]:
net = U2NETP(3,1)

state_dict=torch.load("u2netp.pth",map_location='cpu')
net.load_state_dict(state_dict)
if torch.cuda.is_available():    net.cuda()
net=net.eval();

In [73]:
my_schema_shape={'category': [{'name': 'car', 'color': '#3c6127', 
                               'properties': [
                                            {"name": 'occluded',    "type": 'checkbox' , "default": False },
                                            { "name": 'model type', "type": 'dropdown' , "enum": ['toyota', 'chery', 'ford'], "default": 'toyota' },
                                            { "name": 'color',      "type": 'textfield', "default": 'gray' }]},
              {'name': 'bicycle', 'color': '#89c2bc', 'properties': []},
              {'name': 'person', 'color': '#971ad2', 'properties': []},
              {'name': 'motorcycle', 'color': '#e7e8d1', 'properties': []},
              {'name': 'airplane', 'color': '#df753a', 'properties': []},
              {'name': 'bus', 'color': '#075c52', 'properties': []},
              {'name': 'train', 'color': '#a0da30', 'properties': []},
              {'name': 'truck', 'color': '#7b0915', 'properties': []},
              {'name': 'boat', 'color': '#2567cd', 'properties': []},
              {'name': 'traffic sign', 'color': '#928520', 'properties': []},
              {'name': 'fire hydrant', 'color': '#459511', 'properties': []},
              {'name': 'stop sign', 'color': '#80e278', 'properties': []}]}

In [74]:
from ipywidgets.widgets import Output

output_infos=Output()
output_size=320
norm=transforms.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225))

def deep_update_shape(change):
    
    annotations=change['new']
    with  output_infos:
        print("deep udpate")
    w,h=image.size
    annotation=annotations[-1]
    
    vertices=np.array(annotation["geometry"]["vertices"])
   
    if  vertices.shape[0]>10*2 : return

    vertices[0:vertices.shape[0]:2]*=w
    vertices[1:vertices.shape[0]:2]*=h

    if annotation["geometry"]["type"]=="polygon" :
        vertices=vertices.reshape((-1,1,2)).astype(np.int32)
        bd_rect=cv2.boundingRect(vertices)
        vertices=np.array([bd_rect[0],bd_rect[1],bd_rect[0]+bd_rect[2],bd_rect[1]+bd_rect[3]])
        vertices=vertices.flatten()

    image_in_pil=image.crop(map(int,vertices))
    w_i,h_i=image_in_pil.size
    #display(image_in_pil)

    image_in=image_in_pil.resize((output_size,output_size),resample=Image.BICUBIC)
    image_in=np.array(image_in).astype(np.float)    

    image_in = torch.Tensor(image_in/255.0).permute(2,0,1).float()
    image_in=norm(image_in)
    image_in=image_in.unsqueeze(0)
    with torch.no_grad():
        output,*_ =net(image_in)
    

    pred = output.detach().numpy() #[:,0,:,:]
    pred=(pred-pred.min())/(pred.max()-pred.min())
    del output,_
    
    pred=(pred*255).astype(np.uint8).squeeze(0)
    pred=Image.fromarray(pred).resize((w_i,h_i),resample=Image.BILINEAR)

    bin_mask=(np.array(pred)>100).astype(np.uint8)
    contours,_ = cv2.findContours(bin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours)>0:
        areas=[cv2.contourArea(cnt) for cnt in contours]
        ind_max=np.argmax(areas)

        cnt=cv2.approxPolyDP(contours[ind_max], 1, closed=True)
        cnt=cnt.squeeze().flatten().astype(float)
        cnt[0:cnt.shape[0]:2]=(cnt[0:cnt.shape[0]:2]+int(vertices[0]))/w
        cnt[1:cnt.shape[0]:2]=(cnt[1:cnt.shape[0]:2]+int(vertices[1]))/h
        annotations[-1]["geometry"]= {"vertices": list(cnt), "type": "polygon"} 

        #display(pred)
        #image_out=(np.array(image_in_pil)*(np.array(pred)[:,:,None]).astype(np.float32)/255).astype(np.uint8)
        #display(Image.fromarray(image_out))
    #with output_infos:
    #    print(annotations)
    w_pix.setAnnotations(annotations)

In [76]:
image=Image.open('/media/nallezard/My Passport1/Sofia_dataset/data/images/OA 435_NDOC177003_Corrosion foisonnante sur le m.jpg')
#image=Image.open('image.jpg')
w_pix = Pixano(element='pxn-polygon',image=image,label_schema=my_schema_shape)


display(w_pix)
display(output_infos)

w_pix.observe(deep_update_shape, names='annotations')



Pixano(value=None, element='pxn-polygon', image='iVBORw0KGgoAAAANSUhEUgAACiAAAAeYCAIAAAAtiRqxAAAp9mVYSWZJSSoAC…

Output(outputs=({'output_type': 'stream', 'text': 'deep udpate\ndeep udpate\ndeep udpate\ndeep udpate\ndeep ud…

/tmp/ipykernel_8521/2888249225.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  image_in=np.array(image_in).astype(np.float)
/tmp/ipykernel_8521/2888249225.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  image_in=np.array(image_in).astype(np.float)
